In [2]:
!pip install tensorflow==2.15.0 --ignore-installed


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for tensorflow==2.15.0 from https://files.pythonhosted.org/packages/93/c0/a774286d0383419f558deb27096e5de9f9facd6c27df8e9f9af6fba2f77e/tensorflow-2.15.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached tensorflow-2.15.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.4 kB)
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/f6/d4/349f7f4bd5ea92dab34f5bb0fe31775ef6c311427a14d5a5b31ecb442341/absl_py-2.2.2-py3-none-any.whl.metadata
  Using cached absl_py-2.2.2-py3-none-any.whl.metadata (2.6 kB)
  Obtaining dependency information for astunparse>=1.6.0 from https://files.pythonhosted.org/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36ec61f7ce6028/astunparse-1.6.3-py2.py3-none-any.whl.metadata
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)

In [3]:
!pip install tensorflow==2.15.0 numpy==1.26.4 matplotlib==3.8.2 scikit-learn==1.4.2 seaborn==0.13.2 pandas==2.2.1 scipy==1.12.0


Defaulting to user installation because normal site-packages is not writeable


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import numpy as np
import os
import zipfile
import random
import shutil
from pathlib import Path

In [4]:
# Define the zip file name and the directory to extract to
zip_path = "american-sign-language-letters-subdir.zip"  # replace with your actual zip filename
extract_to = "data"  # or any folder name you want

import os
print(os.getcwd())

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"Extracted '{zip_path}' to '{extract_to}/'")

/sfs/gpfs/tardis/home/mwm6nq/DS4002
Extracted 'american-sign-language-letters-subdir.zip' to 'data/'


In [5]:
# Original dataset folder
SOURCE_DIR = "data/subdir"
# Output directories
TRAIN_VAL_DIR = "data/train_val"
TEST_DIR = "data/test"
# Test split ratio
TEST_SPLIT = 0.15  # 15% of images per class go to test

# Create target dirs if they don't exist
os.makedirs(TRAIN_VAL_DIR, exist_ok=True)
os.makedirs(TEST_DIR, exist_ok=True)

# Seed for reproducibility
random.seed(42)

# Loop through each class folder (A, B, C, ...)
for class_name in os.listdir(SOURCE_DIR):
    class_path = os.path.join(SOURCE_DIR, class_name)
    if not os.path.isdir(class_path):
        continue

    # List image files in the class folder
    image_files = [f for f in os.listdir(class_path) if f.lower().endswith((".jpg"))]
    random.shuffle(image_files)

    # Calculate how many to move to test
    test_count = int(len(image_files) * TEST_SPLIT)
    print(test_count)

    # Define class folders in new dirs
    train_val_class_dir = os.path.join(TRAIN_VAL_DIR, class_name)
    test_class_dir = os.path.join(TEST_DIR, class_name)
    os.makedirs(train_val_class_dir, exist_ok=True)
    os.makedirs(test_class_dir, exist_ok=True)

    # Move files to test and train_val
    for i, file_name in enumerate(image_files):
        src_path = os.path.join(class_path, file_name)
        if i < test_count:
            dst_path = os.path.join(test_class_dir, file_name)
        else:
            dst_path = os.path.join(train_val_class_dir, file_name)
        shutil.copy2(src_path, dst_path)

print(test_count)


8
10
8
9
10
10
8
10
9
9
11
10
9
10
9
9
8
9
13
7
7
12
11
11
9
11
11


In [7]:
#image information
data_dir = "data/subdir"
img_height, img_width = 64, 64
batch_size = 32

In [8]:
# Load raw images and split, better practice to clean after split

# Train/validation split from train_val folder
train_val_dir = "data/train_val"
test_dir = "data/test"

# Training set
raw_train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_val_dir,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode="int"
)

# Validation set
raw_val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_val_dir,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode="int"
)

# Test set — loaded from separate folder
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode="int"
)

Found 1482 files belonging to 26 classes.
Using 1186 files for training.
Found 1482 files belonging to 26 classes.
Using 296 files for validation.
Found 247 files belonging to 26 classes.


In [9]:
# Preprocessing function: grayscale + normalize scales image of pixels
def preprocess(image, label):
    image = tf.image.rgb_to_grayscale(image)
    image = tf.cast(image, tf.float32) / 255.0
    return image, label


In [10]:
# Apply preprocessing to train and test ds
train_ds = raw_train_ds.map(preprocess)
val_ds = raw_val_ds.map(preprocess)
test_ds = raw_val_ds.map(preprocess)

In [11]:
#uses float16 for fast training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SeparableConv2D, BatchNormalization, GlobalAveragePooling2D, Dense, Dropout, Input
#builds a lightweight convolutional neural network (CNN)

#defines the shape of the model, a 64x64 grayscale 
# the model then takes each ASL sign image and extracts features using efficient convolutional layers
#Reduces dimensions of image using global pooling
#Learns final patterns with Dense layers
#outputs a probability for each letter (A–Z + space)
model = Sequential([
    Input(shape=(64, 64, 1)),  # 1 channel because grayscale

    SeparableConv2D(32, 3, activation='relu'), #building a light weight model
    BatchNormalization(), #normalizes the outputs
    SeparableConv2D(64, 3, activation='relu'),
    BatchNormalization(),
    GlobalAveragePooling2D(),

    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(27, activation='softmax', dtype='float32')  # 26 letters + maybe 1 for "space"
])


In [13]:
#compile the model

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 separable_conv2d_2 (Separa  (None, 62, 62, 32)        73        
 bleConv2D)                                                      
                                                                 
 batch_normalization_2 (Bat  (None, 62, 62, 32)        128       
 chNormalization)                                                
                                                                 
 separable_conv2d_3 (Separa  (None, 60, 60, 64)        2400      
 bleConv2D)                                                      
                                                                 
 batch_normalization_3 (Bat  (None, 60, 60, 64)        256       
 chNormalization)                                                
                                                                 
 global_average_pooling2d_1  (None, 64)               

In [14]:
#train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15
)

Epoch 1/15
38/38 [==============================] - 121s 3s/step - loss: 3.3383 - accuracy: 0.0371 - val_loss: 3.2937 - val_accuracy: 0.0338
Epoch 2/15
38/38 [==============================] - 116s 3s/step - loss: 3.2681 - accuracy: 0.0590 - val_loss: 3.2917 - val_accuracy: 0.0372
Epoch 3/15
38/38 [==============================] - 116s 3s/step - loss: 3.2477 - accuracy: 0.0616 - val_loss: 3.2890 - val_accuracy: 0.0372
Epoch 4/15
38/38 [==============================] - 117s 3s/step - loss: 3.2197 - accuracy: 0.0616 - val_loss: 3.2869 - val_accuracy: 0.0372
Epoch 5/15
38/38 [==============================] - 116s 3s/step - loss: 3.2068 - accuracy: 0.0759 - val_loss: 3.2847 - val_accuracy: 0.0541
Epoch 6/15
38/38 [==============================] - 117s 3s/step - loss: 3.1889 - accuracy: 0.0885 - val_loss: 3.2838 - val_accuracy: 0.0676
Epoch 7/15
38/38 [==============================] - 116s 3s/step - loss: 3.1716 - accuracy: 0.0877 - val_loss: 3.2832 - val_accuracy: 0.0507
Epoch 8/15
38

In [16]:
#evaluate on test set
test_loss, test_acc = model.evaluate(test_ds)
print(f"✅ Final Test Accuracy: {test_acc:.4f}")


10/10 [==============================] - 3s 293ms/step - loss: 3.2110 - accuracy: 0.0642
✅ Final Test Accuracy: 0.0642
